<a href="https://colab.research.google.com/github/gulizsamgar/PYTHON-Customer-Segmentation-with-RFM/blob/main/FLO_RFM_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **FLO Müşteri Segmentasyonu (RFM Analizi)**

* * *

*Proje Yazarı: Güliz Samğar*

*Son Güncelleme Tarihi: 18 Ağustos 2025*

* * *

**Proje Genel Bakış**

Bu proje kapsamında, FLO’nun 20.000 müşteriden oluşan veri seti üzerinde RFM (Recency, Frequency, Monetary) analizi uygulanmıştır. Çalışmanın amacı, müşterilerin alışveriş davranışlarını analiz ederek sadakat seviyelerini ölçmek, müşteri segmentasyonu yapmak ve hedef pazarlama stratejileri oluşturmaktır. FLO müşterilerini segmentlere ayırıp bu segmentlere göre pazarlama stratejileri belirlemek istiyor. Buna yönelik olarak müşterilerin davranışları tanımlanacak ve bu davranış öbeklenmelerine göre gruplar oluşturulacak.

Proje Adımları:

1.  **Veriyi Tanıma ve Hazırlama:** Veri setinin yüklenmesi, genel bilgilerin incelenmesi, eksik değer kontrolü ve yeni değişkenlerin (toplam sipariş, toplam harcama) oluşturulması.
2.  **RFM Metriklerinin Hesaplanması:** Recency (Son siparişin üzerinden geçen süre), Frequency (Toplam sipariş sayısı) ve Monetary (Toplam harcama) değerlerinin hesaplanması.
3.  **RF ve RFM Skorlarının Hesaplanması:** Hesaplanan RFM metriklerinin 1-5 arası skorlara dönüştürülmesi ve RF/RFM skorlarının oluşturulması.
4.  **Segmentlerin Oluşturulması:** RF skorlarına göre müşterilerin önceden tanımlanmış segmentlere atanması.
5.  **Eyleme Dönüştürülebilir İçgörüler:** Oluşturulan segmentlerin özelliklerinin incelenmesi ve belirli pazarlama kampanyaları için hedef müşteri kitlelerinin belirlenmesi.
6.  **BONUS: Tüm Süreci Fonksiyonlaştırma:** RFM segmentasyon adımlarının tekrar kullanılabilir bir fonksiyon içine alınması.

Bu analiz sonucunda elde edilen müşteri segmentleri, FLO'nun pazarlama faaliyetlerini daha etkili ve kişiselleştirilmiş hale getirmesine yardımcı olacaktır.

Bu projede kullanılan veri setine [Github'dan](https://github.com/gulizsamgar/PYTHON-Customer-Segmentation-with-RFM/blob/b42d687f63dc9538bf89e7b4384bfeb8670ecd8b/dataset/flo_data_20k.csv) ulaşabilirsiniz.

> *Veri setimizle çalışmaya hazırız. Haydi başlayalım!*



***

**Kütüphanelerin İçe Aktarılması ve Görüntü Ayarları**

Bu bölüm, veri analizi için gerekli kütüphaneleri içe aktarır ve pandas görüntü ayarlarını yapılandırır.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import datetime as dt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.width',1000)

---

**GÖREV 1: VERİYİ TANIMA VE HAZIRLAMA**


In [ ]:
###############################################################
# 1. Veri setini içe aktar
###############################################################

# Dosya yolu tanımlanıyor.
file_path = '/content/drive/MyDrive/flo_data_20k.csv'
# CSV dosyasını pandas DataFrame'ine oku.
df_ = pd.read_csv(file_path)
# Orijinal DataFrame'in bir kopyasını oluştur.
df = df_.copy()

# Doğru şekilde yüklendiğini onaylamak için ilk birkaç satırı görüntüle
print("\nFirst 5 rows of dataset:")
print(df.head(), "\n")


First 5 rows of dataset:
                              master_id order_channel last_order_channel first_order_date last_order_date last_order_date_online last_order_date_offline  order_num_total_ever_online  order_num_total_ever_offline  customer_value_total_ever_offline  customer_value_total_ever_online       interested_in_categories_12
0  cc294636-19f0-11eb-8d74-000d3a38a36f   Android App            Offline       2020-10-30      2021-02-26             2021-02-21              2021-02-26                         4.00                          1.00                             139.99                            799.38                           [KADIN]
1  f431bd5a-ab7b-11e9-a2fc-000d3a38a36f   Android App             Mobile       2017-02-08      2021-02-16             2021-02-16              2020-01-10                        19.00                          2.00                             159.97                           1853.58  [ERKEK, COCUK, KADIN, AKTIFSPOR]
2  69b69676-1a40-11ea-941b-00

  **Veri Seti Bilgisi**

  Bu veri seti, 2020 - 2021 yıllarında OmniChannel (hem online hem offline alışveriş yapan) olarak alışveriş yapan müşterilerin geçmiş alışveriş davranışlarından elde edilen bilgilerden oluşmaktadır.

  Veri setindeki sütunlar aşağıdaki bilgileri içermektedir:

*   **master\_id**: Eşsiz müşteri numarası
*   **order\_channel**: Alışveriş yapılan platforma ait hangi kanalın kullanıldığı (Android, ios, Desktop, Mobile, Offline)
*   **last\_order\_channel**: En son alışverişin yapıldığı kanal
*   **first\_order\_date**: Müşterinin yaptığı ilk alışveriş tarihi
*   **last\_order\_date**: Müşterinin yaptığı son alışveriş tarihi
*   **last\_order\_date\_online**: Müşterinin online platformda yaptığı son alışveriş tarihi
*   **last\_order\_date\_offline**: Müşterinin offline platformda yaptığı son alışveriş tarihi
*   **order\_num\_total\_ever\_online**: Müşterinin online platformda yaptığı toplam alışveriş sayısı
*   **order\_num\_total\_ever\_offline**: Müşterinin offline'da yaptığı toplam alışveriş sayısı
*   **customer\_value\_total\_ever\_offline**: Müşterinin offline alışverişlerinde ödediği toplam ücret
*   **customer\_value\_total\_ever\_online**: Müşterinin online alışverişlerinde ödediği toplam ücret
*   **interested\_in\_categories\_12**: Müşterinin son 12 ayda alışveriş yaptığı kategorilerin listesi

---

In [ ]:
###############################################################
# 2. Veri keşfi
###############################################################

# Veri setinin ilk 10 satırını yazdır.
print("\nFirst 10 rows:")
print(df.head(10), "\n")

# Sütun isimlerini yazdır.
print("\nColumn names:")
print(df.columns, "\n")

# Veri setinin boyutlarını (satır ve sütun sayısı) yazdır.
print("\nShape of dataset (rows, columns):", df.shape, "\n")

# Betimsel istatistikleri (sayısal sütunlar için) yazdır.
print("\nDescriptive statistics:")
print(df.describe().T, "\n")

# Her sütundaki eksik (NaN) değerlerin sayısını yazdır.
print("\nMissing values by column:")
print(df.isnull().sum(), "\n")

# Veri setinin sütunları, eksik olmayan değer sayısı ve veri tipleri hakkında bilgi yazdır.
print("\nData types:")
print(df.info(), "\n")


First 10 rows:
                              master_id order_channel last_order_channel first_order_date last_order_date last_order_date_online last_order_date_offline  order_num_total_ever_online  order_num_total_ever_offline  customer_value_total_ever_offline  customer_value_total_ever_online       interested_in_categories_12
0  cc294636-19f0-11eb-8d74-000d3a38a36f   Android App            Offline       2020-10-30      2021-02-26             2021-02-21              2021-02-26                         4.00                          1.00                             139.99                            799.38                           [KADIN]
1  f431bd5a-ab7b-11e9-a2fc-000d3a38a36f   Android App             Mobile       2017-02-08      2021-02-16             2021-02-16              2020-01-10                        19.00                          2.00                             159.97                           1853.58  [ERKEK, COCUK, KADIN, AKTIFSPOR]
2  69b69676-1a40-11ea-941b-000d3a38a36f

---

In [ ]:
###############################################################
# 3. Özellik Mühendisliği: Müşteri bazında toplam sipariş sayısı ve toplam harcama
###############################################################

df["order_num_total"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
df["customer_value_total"] = df["customer_value_total_ever_offline"] + df["customer_value_total_ever_online"]


**Yeni Değişkenlerin Oluşturulması**

Bu adımda, herbir müşterinin toplam sipariş sayısını ve toplam harcama değerini hesaplamak için iki yeni değişken oluşturulmuştur:

*   `order_num_total`: Müşterinin hem online hem de offline platformlarda yaptığı toplam sipariş sayısıdır. Online ve offline sipariş sayıları toplanarak elde edilir.
*   `customer_value_total`: Müşterinin hem online hem de offline alışverişlerinde ödediği toplam ücrettir. Online ve offline harcama değerleri toplanarak elde edilir.

---

In [ ]:
###############################################################
# 4. Tarih değişkenlerini datetime tipine çevir
###############################################################

# Sütun isimleri içinde "date" geçenleri seç.
date_columns = df.columns[df.columns.str.contains("date")]
# Seçilen sütunları datetime tipine çevir.
df[date_columns] = df[date_columns].apply(pd.to_datetime)

**Tarih Değişkenlerinin Dönüştürülmesi**

Bu adımda, veri setindeki tarih bilgilerini içeren sütunlar (`first_order_date`, `last_order_date`, `last_order_date_online`, `last_order_date_offline`) pandas'ın `datetime` tipine dönüştürülmüştür.

Bu dönüşüm, tarihler üzerinde kolayca hesaplamalar yapabilmemizi sağlar. Örneğin, iki tarih arasındaki farkı bulmak (recency hesaplaması için gerekli), tarihleri belirli formatlarda görüntülemek veya zaman serisi analizleri yapmak için `datetime` tipi gereklidir. Bu sayede tarih verilerini daha etkin bir şekilde kullanabiliriz.

---

In [ ]:
###############################################################
# 5. Kanal bazında dağılım
###############################################################

# order_channel sütununa göre gruplama yap.
print("\nDistribution by order channel:")
# Her kanal için müşteri sayısı, toplam sipariş sayısı),toplam harcama hesapla.
print(df.groupby("order_channel").agg({"master_id": "count",
                                       "order_num_total": "sum",
                                       "customer_value_total": "sum"}), "\n")


Distribution by order channel:
               master_id  order_num_total  customer_value_total
order_channel                                                  
Android App         9495         52269.00            7819062.76
Desktop             2735         10920.00            1610321.46
Ios App             2833         15351.00            2525999.93
Mobile              4882         21679.00            3028183.16 



**Kanal Bazında Dağılım Analizi**

Bu adımda, müşterilerin hangi kanalları (Android App, Desktop, Ios App, Mobile) kullanarak alışveriş yaptıkları incelenmiştir. `order_channel` sütununa göre gruplama yapılarak her bir kanal için aşağıdaki metrikler hesaplanmıştır:

*   **master\_id (count)**: O kanalı kullanan eşsiz müşteri sayısı.
*   **order\_num\_total (sum)**: O kanal üzerinden yapılan toplam sipariş sayısı.
*   **customer\_value\_total (sum)**: O kanal üzerinden yapılan toplam harcama değeri.

Elde edilen sonuçlar, hangi kanalların daha fazla müşteriye, daha yüksek sipariş sayısına veya daha yüksek toplam harcamaya sahip olduğunu göstererek pazarlama stratejilerinin hangi kanallara odaklanması gerektiği konusunda bilgi sağlar.

---

In [ ]:
###############################################################
# 6. En çok gelir getiren ilk 10 müşteri
###############################################################

# "customer_value_total" sütununa göre DataFrame'i azalan sırada sırala.
# İlk 10 satırı seç.
print("\nTop 10 customers by total revenue:")
print(df.sort_values("customer_value_total", ascending=False).head(10), "\n")


Top 10 customers by total revenue:
                                  master_id order_channel last_order_channel first_order_date last_order_date last_order_date_online last_order_date_offline  order_num_total_ever_online  order_num_total_ever_offline  customer_value_total_ever_offline  customer_value_total_ever_online                   interested_in_categories_12  order_num_total  customer_value_total
11150  5d1c466a-9cfd-11e9-9897-000d3a38a36f   Android App            Desktop       2013-10-11      2021-04-30             2021-04-30              2020-12-24                       200.00                          2.00                             684.97                          45220.13  [AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]           202.00              45905.10
4315   d5ef8058-a5c6-11e9-a2fc-000d3a38a36f   Android App        Android App       2018-08-06      2021-02-23             2021-02-23              2020-07-06                        67.00                          1.00         

**En Çok Gelir Getiren İlk 10 Müşteri**

Bu adımda, `customer_value_total` sütununa göre DataFrame'i azalan sırada sıralayarak (en yüksek toplam harcamadan en düşüğe doğru) en çok gelir getiren ilk 10 müşteriyi belirledik.

Bu liste, şirketin en değerli müşterilerini hızlıca görmemizi sağlar. Bu müşterilere özel pazarlama kampanyaları, sadakat programları veya ayrıcalıklar sunarak ilişkileri güçlendirmek ve daha fazla gelir elde etmek hedeflenebilir.

---

In [ ]:
###############################################################
# 7. En çok sipariş veren ilk 10 müşteri
###############################################################

# "order_num_total" sütununa göre DataFrame'i azalan sırada sırala.
# İlk 10 satırı seç.
print("\nTop 10 customers by total number of orders:")
print(df.sort_values("order_num_total", ascending=False).head(10), "\n")


Top 10 customers by total number of orders:
                                  master_id order_channel last_order_channel first_order_date last_order_date last_order_date_online last_order_date_offline  order_num_total_ever_online  order_num_total_ever_offline  customer_value_total_ever_offline  customer_value_total_ever_online                   interested_in_categories_12  order_num_total  customer_value_total
11150  5d1c466a-9cfd-11e9-9897-000d3a38a36f   Android App            Desktop       2013-10-11      2021-04-30             2021-04-30              2020-12-24                       200.00                          2.00                             684.97                          45220.13  [AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]           202.00              45905.10
7223   cba59206-9dd1-11e9-9897-000d3a38a36f   Android App        Android App       2013-02-21      2021-05-09             2021-05-09              2020-01-25                       130.00                          1.00

**En Çok Sipariş Veren İlk 10 Müşteri**

Bu adımda, `order_num_total` sütununa göre DataFrame'i azalan sırada sıralayarak (en yüksek toplam sipariş sayısından en düşüğe doğru) en çok sipariş veren ilk 10 müşteriyi belirledik.

Bu liste, şirketin en sadık ve sık alışveriş yapan müşterilerini görmemizi sağlar. Bu müşterilere yönelik özel teklifler, erken erişim kampanyaları veya topluluk etkinlikleri düzenleyerek müşteri bağlılığını artırmak hedeflenebilir.

---

---

**GÖREV 2: RFM METRİKLERİNİN HESAPLANMASI**

Bu bölüm, RFM (Recency, Frequency, Monetary) analizi için gerekli olan temel metriklerin hesaplanmasını içerir.

In [ ]:
###############################################################
# 1. Analiz tarihi: Son alışveriş tarihinden 2 gün sonrası
###############################################################

# Veri setindeki son sipariş tarihini bul.
print("\nLast order date in dataset:")
print(df["last_order_date"].max())  # Veri setindeki son tarih 2021-05-30

# Analiz tarihini belirle (son sipariş tarihinden 2 gün sonrası).
analysis_date = dt.datetime(2021, 6, 1)

###############################################################
# 2. RFM dataframe oluşturma
###############################################################

# RFM metriklerini saklamak için boş bir DataFrame oluştur.
rfm = pd.DataFrame()
# Müşteri ID'lerini RFM DataFrame'ine kopyala.
rfm["customer_id"] = df["master_id"]

# Recency (Yenilik): Analiz tarihi ile son sipariş tarihi arasındaki gün farkını hesapla.
rfm["recency"] = (analysis_date - df["last_order_date"]).dt.days

# Frequency (Sıklık): Toplam sipariş sayısını kullan.
rfm["frequency"] = df["order_num_total"]

# Monetary (Parasal Değer): Toplam harcama değerini kullan.
rfm["monetary"] = df["customer_value_total"]

# Oluşturulan RFM DataFrame'inin ilk 5 satırını yazdır.
print("\nFirst 5 rows of RFM dataframe:")
print(rfm.head(), "\n")


Last order date in dataset:
2021-05-30 00:00:00

First 5 rows of RFM dataframe:
                            customer_id  recency  frequency  monetary
0  cc294636-19f0-11eb-8d74-000d3a38a36f       95       5.00    939.37
1  f431bd5a-ab7b-11e9-a2fc-000d3a38a36f      105      21.00   2013.55
2  69b69676-1a40-11ea-941b-000d3a38a36f      186       5.00    585.32
3  1854e56c-491f-11eb-806e-000d3a38a36f      135       2.00    121.97
4  d6ea1074-f1f5-11e9-9346-000d3a38a36f       86       2.00    209.98 



**RFM Metriklerinin Hesaplanması ve RFM DataFrame'inin Oluşturulması**

Bu adımda, RFM analizi için temel olan Recency, Frequency ve Monetary metrikleri hesaplanarak `rfm` adında yeni bir DataFrame oluşturulmuştur.

1.  **Recency (Yenilik):** Müşterinin son alışveriş tarihi ile analiz tarihi arasındaki gün farkı hesaplanır. Analiz tarihi olarak, veri setindeki en son sipariş tarihinden 2 gün sonrası (`2021-06-01`) belirlenmiştir. Recency değeri düşük olan müşteriler daha yakın zamanda alışveriş yapmışlardır ve genellikle daha değerli olarak kabul edilirler.
2.  **Frequency (Sıklık):** Müşterinin toplam sipariş sayısı (`order_num_total`) Frequency metriği olarak kullanılır. Frequency değeri yüksek olan müşteriler daha sık alışveriş yapmaktadırlar.
3.  **Monetary (Parasal Değer):** Müşterinin toplam harcama değeri (`customer_value_total`) Monetary metriği olarak kullanılır. Monetary değeri yüksek olan müşteriler şirkete daha fazla gelir sağlamışlardır.

Bu metrikler, her bir müşteri için hesaplanarak yeni `rfm` DataFrame'ine `customer_id`, `recency`, `frequency` ve `monetary` sütunları olarak eklenir. Bu DataFrame, müşteri segmentasyonu için temel oluşturacaktır.

---

---

**GÖREV 3: RF VE RFM SKORLARININ HESAPLANMASI**

Bu bölüm, RF ve RFM (Recency, Frequency, Monetary) skorlarının hesaplanmasını içerir.

In [ ]:
###############################################################
# 1. Recency, Frequency, Monetary skorlarının hesaplanması
###############################################################

# Recency skorunu hesapla: Değerleri 5 gruba ayır ve 5 en iyi (en düşük recency) olacak şekilde etiketle.
rfm["recency_score"] = pd.qcut(rfm["recency"], 5, labels=[5, 4, 3, 2, 1])
# Frequency skorunu hesapla: Değerlerin rank'ını al, 5 gruba ayır ve 5 en iyi (en yüksek frequency) olacak şekilde etiketle.
rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
# Monetary skorunu hesapla: Değerleri 5 gruba ayır ve 5 en iyi (en yüksek monetary) olacak şekilde etiketle.
rfm["monetary_score"] = pd.qcut(rfm["monetary"], 5, labels=[1, 2, 3, 4, 5])

###############################################################
# 2. RF_SCORE oluşturma
###############################################################

# Recency ve Frequency skorlarını birleştirerek RF_SCORE oluştur.
rfm["RF_SCORE"] = (rfm["recency_score"].astype(str) +
                   rfm["frequency_score"].astype(str))

###############################################################
# 3. RFM_SCORE oluşturma
###############################################################

# Recency, Frequency ve Monetary skorlarını birleştirerek RFM_SCORE oluştur.
rfm["RFM_SCORE"] = (rfm["recency_score"].astype(str) +
                    rfm["frequency_score"].astype(str) +
                    rfm["monetary_score"].astype(str))

# RFM skorları eklenmiş DataFrame'in ilk 5 satırını yazdır.
print("\nFirst 5 rows of RFM with scores:\n")
print(rfm.head())


First 5 rows of RFM with scores:

                            customer_id  recency  frequency  monetary recency_score frequency_score monetary_score RF_SCORE RFM_SCORE
0  cc294636-19f0-11eb-8d74-000d3a38a36f       95       5.00    939.37             3               4              4       34       344
1  f431bd5a-ab7b-11e9-a2fc-000d3a38a36f      105      21.00   2013.55             3               5              5       35       355
2  69b69676-1a40-11ea-941b-000d3a38a36f      186       5.00    585.32             2               4              3       24       243
3  1854e56c-491f-11eb-806e-000d3a38a36f      135       2.00    121.97             3               1              1       31       311
4  d6ea1074-f1f5-11e9-9346-000d3a38a36f       86       2.00    209.98             3               1              1       31       311


**RF ve RFM Skorlarının Hesaplanması**

Bu adımda, daha önce hesapladığımız Recency, Frequency ve Monetary metrikleri 1 ile 5 arasında değişen skorlara dönüştürülmüş ve bu skorlar birleştirilerek RF ve RFM Skorları oluşturulmuştur.

1.  **Skorlama:** Her bir metrik (`recency`, `frequency`, `monetary`) `pd.qcut()` fonksiyonu kullanılarak beş eşit parçaya (quantiles) bölünmüştür.
    *   **Recency Skor:** En düşük recency değerine sahip (en son alışveriş yapan) müşteriler en yüksek skoru (5) alacak şekilde tersine skorlanmıştır (`labels=[5, 4, 3, 2, 1]`). Bu tersine skorlama sayesinde, düşük recency değerleri yüksek skorlarla eşleşir.
    *   **Frequency Skor:** En yüksek frequency değerine sahip (en sık alışveriş yapan) müşteriler en yüksek skoru (5) alacak şekilde skorlanmıştır. Aynı frequency değerine sahip müşteriler arasında tutarlı bir sıralama sağlamak ve `qcut`'ın doğru çalışması için `rank(method="first")` kullanılarak değerler önce sıralanmıştır.
    *   **Monetary Skor:** En yüksek monetary değerine sahip (en çok harcama yapan) müşteriler en yüksek skoru (5) alacak şekilde skorlanmıştır (`labels=[1, 2, 3, 4, 5]`).
2.  **RF\_SCORE Oluşturma:** Recency ve Frequency skorları string'e çevrilerek birleştirilmiştir (Örn: Recency=5, Frequency=4 -> RF\_SCORE="54").
3.  **RFM\_SCORE Oluşturma:** Recency, Frequency ve Monetary skorları string'e çevrilerek birleştirilmiştir (Örn: Recency=5, Frequency=4, Monetary=5 -> RFM\_SCORE="545").

Bu skorlar ve birleşik skorlar, müşterileri farklı segmentlere ayırmak için kullanılacaktır.

---

---

**GÖREV 4: SEGMENTLERİN OLUŞTURULMASI**

Bu bölüm, hesaplanan RF skorları üzerinden müşteri segmentlerinin oluşturulmasını içerir.

In [ ]:
###############################################################
# 1. Segment tanımlamaları
###############################################################

# RF skorlarına karşılık gelen segment isimlerini tanımlayan bir sözlük oluştur.
# Anahtarlar regex desenleri, değerler ise segment isimleridir.
seg_map = {
    r"[1-2][1-2]": "hibernating", # Düşük Recency ve Düşük Frequency
    r"[1-2][3-4]": "at_Risk",     # Düşük Recency, Orta-Yüksek Frequency
    r"[1-2]5": "cant_loose",      # Düşük Recency, Yüksek Frequency
    r"3[1-2]": "about_to_sleep",  # Orta Recency, Düşük Frequency
    r"33": "need_attention",      # Orta Recency, Orta Frequency
    r"[3-4][4-5]": "loyal_customers", # Orta-Yüksek Recency, Yüksek Frequency
    r"41": "promising",           # Yüksek Recency, Düşük Frequency (Yeni olabilirler veya tek seferlik)
    r"51": "new_customers",       # En Yüksek Recency, Düşük Frequency (Yeni müşteriler)
    r"[4-5][2-3]": "potential_loyalists", # Yüksek Recency, Orta Frequency
    r"5[4-5]": "champions"        # En Yüksek Recency, En Yüksek Frequency
}

# rfm DataFrame'ine "segment" adında yeni bir sütun ekle.
# RF_SCORE sütunundaki değerleri seg_map sözlüğünü kullanarak segment isimleriyle değiştir.
# regex=True parametresi, anahtarların regex desenleri olarak kullanılmasını sağlar.
rfm["segment"] = rfm["RF_SCORE"].replace(seg_map, regex=True)

# Segment bilgisi eklenmiş DataFrame'in ilk 5 satırını yazdır.
print("\nFirst 5 rows of RFM with segments:\n")
print(rfm.head())


First 5 rows of RFM with segments:

                            customer_id  recency  frequency  monetary recency_score frequency_score monetary_score RF_SCORE RFM_SCORE          segment
0  cc294636-19f0-11eb-8d74-000d3a38a36f       95       5.00    939.37             3               4              4       34       344  loyal_customers
1  f431bd5a-ab7b-11e9-a2fc-000d3a38a36f      105      21.00   2013.55             3               5              5       35       355  loyal_customers
2  69b69676-1a40-11ea-941b-000d3a38a36f      186       5.00    585.32             2               4              3       24       243          at_Risk
3  1854e56c-491f-11eb-806e-000d3a38a36f      135       2.00    121.97             3               1              1       31       311   about_to_sleep
4  d6ea1074-f1f5-11e9-9346-000d3a38a36f       86       2.00    209.98             3               1              1       31       311   about_to_sleep


**RF Skorlarının Segment Olarak Tanımlanması**

Bu adımda, daha önce hesapladığımız RF\_SCORE'ları kullanarak müşterileri önceden tanımlanmış segmentlere ayırdık. RF\_SCORE, müşterinin Recency ve Frequency skorlarının (1'den 5'e kadar olan sayılar) birleştirilmesiyle elde edilen iki haneli bir string'dir (Örn: "54").

1.  **Segment Tanımları:** `seg_map` adında bir sözlük oluşturuldu. Bu sözlüğün anahtarları, RF\_SCORE değerleriyle eşleşen **düzenli ifadeler (regex)** içerir. Örneğin:
    *   `r"[1-2][1-2]"` anahtarı, Recency skoru 1 veya 2 OLAN ve Frequency skoru 1 veya 2 OLAN müşterileri eşleştirir. Bu müşteriler "hibernating" segmentine atanır.
    *   `r"5[4-5]"` anahtarı, Recency skoru 5 OLAN ve Frequency skoru 4 veya 5 OLAN müşterileri eşleştirir. Bu müşteriler "champions" segmentine atanır.
    *   Sözlüğün değerleri ise bu regex desenlerine karşılık gelen segment isimleridir.
2.  **Segment Atama:** `rfm` DataFrame'ine "segment" adında yeni bir sütun eklenmiştir. `rfm["RF_SCORE"].replace(seg_map, regex=True)` kodu kullanılarak, her müşterinin `RF_SCORE` değeri `seg_map` sözlüğündeki regex desenleriyle karşılaştırılır. Eşleşen desenin karşılığı olan segment ismi o müşterinin "segment" sütununa atanır. `regex=True` parametresi, anahtarların düzenli ifade olarak işlenmesini sağlar.

Bu adım sonucunda, her müşteri Recency ve Frequency davranışlarına göre belirli bir segmente atanmış olur. Bu segmentler, hedefli pazarlama kampanyaları ve stratejileri geliştirmek için kullanılacaktır.

---

---

**GÖREV 5: EYLEME DÖNÜŞTÜRÜLEBİLİR İÇGÖRÜLER**

Bu bölüm, RFM analizi ve müşteri segmentasyonundan elde edilen bulguları eyleme dönüştürülebilir içgörülere çevirmeyi amaçlar.

In [ ]:
###############################################################
# 1. Segmentlerin recency, frequency, monetary ortalamaları
###############################################################

print("\nSegment statistics (mean values and counts):\n")
# RFM DataFrame'inden segment, recency, frequency ve monetary sütunlarını seç.
# "segment" sütununa göre gruplama yap.
# Her grup için recency, frequency ve monetary sütunlarının ortalamasını (mean) ve sayısını (count) hesapla.
print(rfm[["segment", "recency", "frequency", "monetary"]]
      .groupby("segment").agg(["mean", "count"]), "\n")


Segment statistics (mean values and counts):

                    recency       frequency       monetary      
                       mean count      mean count     mean count
segment                                                         
about_to_sleep       113.79  1629      2.40  1629   359.01  1629
at_Risk              241.61  3131      4.47  3131   646.61  3131
cant_loose           235.44  1200     10.70  1200  1474.47  1200
champions             17.11  1932      8.93  1932  1406.63  1932
hibernating          247.95  3604      2.39  3604   366.27  3604
loyal_customers       82.59  3361      8.37  3361  1216.82  3361
need_attention       113.83   823      3.73   823   562.14   823
new_customers         17.92   680      2.00   680   339.96   680
potential_loyalists   37.16  2938      3.30  2938   533.18  2938
promising             58.92   647      2.00   647   335.67   647 



**Segment İstatistikleri: Ortalama Değerler ve Müşteri Sayıları**

Bu adımda, oluşturduğumuz müşteri segmentlerinin temel RFM metrikleri (Recency, Frequency, Monetary) açısından ne durumda olduğunu görmek için segment bazında ortalama değerleri ve her segmentteki müşteri sayısını hesapladık.

`rfm[["segment", "recency", "frequency", "monetary"]].groupby("segment").agg(["mean", "count"])` kodu:

1.  `rfm` DataFrame'inden ilgili sütunları (`segment`, `recency`, `frequency`, `monetary`) seçer.
2.  Veriyi `segment` sütununa göre gruplar.
3.  Her bir segment grubu için `recency`, `frequency` ve `monetary` sütunlarının ortalamasını (`mean`) ve o segmentteki müşteri sayısını (`count`) hesaplar.

Bu istatistikler, her bir segmentin özelliklerini anlamamıza yardımcı olur. Örneğin, "champions" segmentinin düşük recency (yakın zamanda alışveriş yapmış), yüksek frequency (sık alışveriş yapmış) ve yüksek monetary (çok harcama yapmış) değerlerine sahip olduğunu; "hibernating" segmentinin ise tam tersi özellikler gösterdiğini görebiliriz. Bu bilgiler, her segmente yönelik özel pazarlama stratejileri geliştirmek için kritik öneme sahiptir.

---

In [ ]:
###############################################################
# 2.a Yeni kadın ayakkabı markası için hedef müşteri seçimi
###############################################################

# "champions" ve "loyal_customers" segmentlerindeki müşteri ID'lerini seç.
target_segments_customer_ids = rfm[rfm["segment"].isin(["champions", "loyal_customers"])]["customer_id"]
# Seçilen segmentlerdeki ve "KADIN" kategorisiyle ilgilenen müşteri ID'lerini filtrele.
cust_ids = df[(df["master_id"].isin(target_segments_customer_ids)) &
              (df["interested_in_categories_12"].str.contains("KADIN"))]["master_id"]

# Hedef müşteri ID'lerini CSV dosyasına kaydet.
cust_ids.to_csv("yeni_marka_hedef_müşteri_id.csv", index=False)
print("\nNew brand target customer IDs saved to yeni_marka_hedef_müşteri_id.csv")
print("Number of customers:", cust_ids.shape[0])


New brand target customer IDs saved to yeni_marka_hedef_müşteri_id.csv
Number of customers: 2497


**Yeni Kadın Ayakkabı Markası İçin Hedef Müşteri Seçimi**

FLO bünyesine yeni bir kadın ayakkabı markası dahil ediyor. Dahil ettiği markanın ürün fiyatları genel müşteri tercihlerinin üstünde. Bu nedenle markanın tanıtımı ve ürün satışları için ilgilenecek profildeki müşterilerle özel olarak iletişime geçilmek isteniliyor. Bu müşterilerin sadık ve kadın kategorisinden alışveriş yapan kişiler olması planlandı.

Bu hedef doğrultusunda, aşağıdaki adımlar izlenerek hedef müşteri kitlesi belirlenmiştir:

1.  **Segment Filtrelemesi:** Öncelikle, sadık müşteri profiline uyan "champions" ve "loyal_customers" segmentlerindeki müşterilerin ID'leri seçilmiştir.
2.  **Kategori Filtrelemesi:** Seçilen bu müşteriler arasından, son 12 ayda "KADIN" kategorisiyle ilgilenenler (alışveriş yapanlar) filtrelenmiştir.
3.  **ID'lerin Kaydedilmesi:** Belirlenen hedef müşteri ID'leri, "yeni\_marka\_hedef\_müşteri\_id.csv" adlı bir CSV dosyasına kaydedilmiştir.

Bu CSV dosyası, yeni markanın tanıtımı ve özel kampanyaları için kullanılacak hedef müşteri listesini içermektedir.

---

In [ ]:
###############################################################
# 2.b Erkek ve Çocuk ürünlerinde indirim için hedef müşteri seçimi
###############################################################

# "cant_loose", "hibernating", "new_customers" segmentlerindeki müşteri ID'lerini seç.
target_segments_customer_ids = rfm[rfm["segment"].isin(["cant_loose", "hibernating", "new_customers"])]["customer_id"]
# Seçilen segmentlerdeki ve "ERKEK" veya "COCUK" kategorileriyle ilgilenen müşteri ID'lerini filtrele.
cust_ids = df[(df["master_id"].isin(target_segments_customer_ids)) &
              ((df["interested_in_categories_12"].str.contains("ERKEK")) |
               (df["interested_in_categories_12"].str.contains("COCUK")))]["master_id"]

# Hedef müşteri ID'lerini CSV dosyasına kaydet.
cust_ids.to_csv("indirim_hedef_müşteri_ids.csv", index=False)
print("\nDiscount campaign target customer IDs saved to indirim_hedef_müşteri_ids.csv")
print("Number of customers:", cust_ids.shape[0])


Discount campaign target customer IDs saved to indirim_hedef_müşteri_ids.csv
Number of customers: 2771


**Erkek ve Çocuk Ürünlerinde İndirim İçin Hedef Müşteri Seçimi**

Erkek ve Çoçuk ürünlerinde %40'a yakın indirim planlanmaktadır. Bu indirimle ilgili kategorilerle ilgilenen geçmişte iyi müşterilerden olan ama uzun süredir alışveriş yapmayan ve yeni gelen müşteriler özel olarak hedef alınmak isteniliyor.

Bu hedef doğrultusunda, aşağıdaki adımlar izlenerek hedef müşteri kitlesi belirlenmiştir:

1.  **Segment Filtrelemesi:** Öncelikle, "cant\_loose" (kaybetmemek gerekenler), "hibernating" (uykuda olanlar) ve "new\_customers" (yeni müşteriler) segmentlerindeki müşterilerin ID'leri seçilmiştir. Bu segmentler, geçmişte değerli olup aktivitesi düşen veya yeni kazanılan müşterileri temsil eder.
2.  **Kategori Filtrelemesi:** Seçilen bu müşteriler arasından, son 12 ayda "ERKEK" veya "COCUK" kategorileriyle ilgilenenler (alışveriş yapanlar) filtrelenmiştir.
3.  **ID'lerin Kaydedilmesi:** Belirlenen hedef müşteri ID'leri, "indirim\_hedef\_müşteri\_ids.csv" adlı bir CSV dosyasına kaydedilmiştir.

Bu CSV dosyası, indirim kampanyası için kullanılacak hedef müşteri listesini içermektedir.

---

---

**BONUS: TÜM SÜRECİ FONKSİYONLAŞTIRMA**

In [ ]:
def create_rfm(dataframe):
    """RFM segmentasyon veri çerçevesi oluşturur"""

    # Veri hazırlama: Toplam sipariş sayısı ve toplam harcama hesapla
    dataframe["order_num_total"] = (dataframe["order_num_total_ever_online"] +
                                    dataframe["order_num_total_ever_offline"])
    dataframe["customer_value_total"] = (dataframe["customer_value_total_ever_offline"] +
                                         dataframe["customer_value_total_ever_online"])

    # Tarih değişkenlerini datetime tipine çevir
    date_columns = dataframe.columns[dataframe.columns.str.contains("date")]
    dataframe[date_columns] = dataframe[date_columns].apply(pd.to_datetime)

    # Analiz tarihi: Son alışveriş tarihinden 2 gün sonrası
    analysis_date = dt.datetime(2021, 6, 1)

    # RFM metrikleri: Recency, Frequency ve Monetary değerlerini hesapla
    rfm = pd.DataFrame()
    rfm["customer_id"] = dataframe["master_id"]
    rfm["recency"] = (analysis_date - dataframe["last_order_date"]).dt.days # Gün farkını hesapla
    rfm["frequency"] = dataframe["order_num_total"] # Toplam sipariş sayısı
    rfm["monetary"] = dataframe["customer_value_total"] # Toplam harcama

    # Skorlar: Recency, Frequency, Monetary skorlarını hesapla ve birleştir
    rfm["recency_score"] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1]) # Recency skoru (ters)
    rfm["frequency_score"] = pd.qcut(rfm['frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5]) # Frequency skoru (rank ile)
    rfm["monetary_score"] = pd.qcut(rfm['monetary'], 5, labels=[1, 2, 3, 4, 5]) # Monetary skoru
    rfm["RF_SCORE"] = (rfm['recency_score'].astype(str) + rfm['frequency_score'].astype(str)) # RF skoru
    rfm["RFM_SCORE"] = (rfm['recency_score'].astype(str) + rfm['frequency_score'].astype(str) + rfm['monetary_score'].astype(str)) # RFM skoru

    # Segmentler: RF_SCORE'a göre müşteri segmentlerini ata
    seg_map = {
        r'[1-2][1-2]': 'hibernating',
        r'[1-2][3-4]': 'at_Risk',
        r'[1-2]5': 'cant_loose',
        r'3[1-2]': 'about_to_sleep',
        r'33': 'need_attention',
        r'[3-4][4-5]': 'loyal_customers',
        r'41': 'promising',
        r'51': 'new_customers',
        r'[4-5][2-3]': 'potential_loyalists',
        r'5[4-5]': 'champions'
    }
    rfm['segment'] = rfm['RF_SCORE'].replace(seg_map, regex=True) # Segment isimlerini ata

    # İstenen sütunları içeren RFM DataFrame'ini döndür
    return rfm[["customer_id", "recency", "frequency", "monetary", "RF_SCORE", "RFM_SCORE", "segment"]]

# Fonksiyonu çalıştır ve sonucu yeni bir DataFrame'e ata
rfm_df = create_rfm(df)
print("\nFinal RFM dataframe created with segments:\n")
print(rfm_df.head(),"\n")


Final RFM dataframe created with segments:

                            customer_id  recency  frequency  monetary RF_SCORE RFM_SCORE          segment
0  cc294636-19f0-11eb-8d74-000d3a38a36f       95       5.00    939.37       34       344  loyal_customers
1  f431bd5a-ab7b-11e9-a2fc-000d3a38a36f      105      21.00   2013.55       35       355  loyal_customers
2  69b69676-1a40-11ea-941b-000d3a38a36f      186       5.00    585.32       24       243          at_Risk
3  1854e56c-491f-11eb-806e-000d3a38a36f      135       2.00    121.97       31       311   about_to_sleep
4  d6ea1074-f1f5-11e9-9346-000d3a38a36f       86       2.00    209.98       31       311   about_to_sleep 



**BONUS: Tüm Süreci Fonksiyonlaştırma**

Bu bölümde, RFM segmentasyon adımlarını tek bir fonksiyon altında toplayan `create_rfm` fonksiyonu oluşturulmuştur.

Bu fonksiyon:

1.  Ham veri setini (DataFrame olarak) girdi olarak alır.
2.  Gerekli veri hazırlıklarını yapar (toplam sipariş ve harcama hesaplama, tarih değişkenlerini dönüştürme).
3.  Belirlenen analiz tarihine göre Recency, Frequency ve Monetary metriklerini hesaplar.
4.  Bu metrikler üzerinden 1-5 arası RFM skorlarını hesaplar.
5.  RF skorlarına göre müşteri segmentlerini belirler.
6.  Sonuç olarak, müşteri ID'leri, RFM metrikleri, RF/RFM skorları ve segment bilgilerini içeren bir DataFrame döndürür.

Bu fonksiyon sayesinde, farklı veri setleri üzerinde kolayca RFM analizi yapabilir ve müşteri segmentasyonu sonuçlarına hızlıca ulaşabilirsiniz. Kodun tekrar kullanılabilirliğini ve okunabilirliğini artırır.

----

----

# Projenin Sonuçları, Faydaları ve Kullanılan Beceriler

**Proje Sonuçları:**

Bu proje kapsamında gerçekleştirilen RFM analizi ve müşteri segmentasyonu ile FLO müşterileri, alışveriş davranışlarına göre anlamlı gruplara ayrılmıştır. Elde edilen segmentler (Champions, Loyal Customers, Hibernating vb.), her bir grubun Recency, Frequency ve Monetary metrikleri açısından farklı özellikler gösterdiğini ortaya koymuştur. Ayrıca, belirli pazarlama kampanyaları için (yeni kadın ayakkabı markası ve Erkek/Çocuk ürünleri indirimi) hedef müşteri listeleri oluşturulmuştur.

---

**Projenin Faydaları:**

Bu segmentasyon çalışmasının FLO'ya sağlayacağı başlıca faydalar şunlardır:

*   **Hedefli Pazarlama:** Farklı müşteri segmentlerine yönelik daha etkili ve kişiselleştirilmiş pazarlama stratejileri geliştirme imkanı.
*   **Müşteri Değeri Yönetimi:** En değerli müşterilerin belirlenerek sadakatlerinin artırılması ve ilişkilerin güçlendirilmesi.
*   **Kaynak Optimizasyonu:** Pazarlama bütçesinin ve çabalarının en verimli şekilde kullanılması.
*   **Karar Alma Süreçleri:** Müşteri davranışlarına dayalı veriye dayalı kararlar alma yeteneği.
*   **Ürün ve Hizmet Geliştirme:** Segmentlerin ilgi alanlarına göre ürün ve hizmetlerin şekillendirilmesi.

---

**Kullanılan Beceriler:**

Bu projeyi tamamlarken aşağıdaki temel beceriler kullanılmıştır:

*   **Veri Okuma ve Anlama:** CSV formatındaki veriyi okuma ve yapısını anlama.
*   **Veri Manipülasyonu ve Hazırlığı:** Pandas kütüphanesi ile veri üzerinde işlemler yapma (toplama, dönüştürme).
*   **Özellik Mühendisliği:** Mevcut verilerden yeni, anlamlı özellikler (toplam sipariş, toplam harcama) türetme.
*   **RFM Analizi:** Recency, Frequency ve Monetary metriklerini hesaplama.
*   **Kantilleme (Quantile Analysis):** Müşterileri RFM metriklerine göre gruplara ayırma (`pd.qcut`).
*   **Skorlama ve Segmentasyon:** RFM skorlarını oluşturma ve segmentasyon kurallarını tanımlama.
*   **Veri Filtreleme ve Seçimi:** Belirli koşullara uyan müşteri alt kümelerini seçme.
*   **Dosya Kaydetme:** Elde edilen sonuçları (hedef müşteri ID'leri) CSV dosyasına kaydetme.
*   **Fonksiyon Yazma:** Tekrar kullanılabilir kod blokları oluşturma.
*   **Temel İstatistiksel Analiz:** Segment ortalamalarını ve sayımlarını hesaplama.